# 배운 걸 응용한 간단한 react agent 제작
### 배운 내용
1. react agent 생성
2. 간단한 tool 제작

In [15]:
!poetry add langchain_openai langgraph python-dotenv langfuse langchain_community langchain_chroma PyMuPDF

The following packages are already present in the pyproject.toml and will be skipped:

  - langchain_openai
  - langgraph
  - python-dotenv
  - langfuse
  - langchain_community
  - langchain_chroma

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Using version ^1.25.3 for pymupdf

Updating dependencies
Resolving dependencies...

Package operations: 1 install, 0 updates, 0 removals

  - Installing pymupdf (1.25.3)

Writing lock file


## 에이전트

### 도구

```python
from langchain.tools import tool
```
해당 라이브러리를 사용해서 사용자 정의의 도구를 만들 수 있다.

사용 방법은 데코레이터 정의처럼 @tool 을 사용하면 agent에 활용할 수 있는 도구가 된다.

### 도구 구현
1. RAG
2. 단순 계산
3. 기타... (미정)

In [17]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import tool
# 문서 로드 - 전처리 - 청크 스플릿 - (임베딩 - 백터스토어 저장)
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

load_dotenv()

True

In [2]:
### 문서 로드
loader = PyMuPDFLoader("./docs/제254호 과학기술&ICT 정책.기술동향.pdf")
document = loader.load()
print(len(document))

68


In [31]:
### 문서 스플릿

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=150)
splited_document = splitter.split_documents(document)

In [32]:
print(splited_document)

[Document(metadata={'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-01-12T12:28:00+09:00', 'source': './docs/제254호 과학기술&ICT 정책.기술동향.pdf', 'file_path': './docs/제254호 과학기술&ICT 정책.기술동향.pdf', 'total_pages': 68, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-01-12T23:13:39+09:00', 'trapped': '', 'modDate': "D:20240112231339+09'00'", 'creationDate': "D:20240112122800+09'00'", 'page': 0}, page_content='No.254\n2024.1.12.\nⅠ이슈 분석:\n1\n2024년 기술 트렌드 및 시사점\nⅡ주요 동향\n17\n1. 과학기술\n미국, 탄소 포집 및 저장(CCS) 현황 보고서 발표\n17\n미국, GCC 국가들의 넷 제로 계획 수립을 위한 권장사항 제시\n19\n일본, ‘산업경쟁력강화법’ 개정에 관한 의견\n22\n일본, ‘바이오 제조’ 추진의 핵심인 ‘Nature positive’ 대응\n24\n독일, 기후 외교 정책에 관한 전략 발표\n26\n독일, ‘연구 혁신 미래 전략’의 실행에 관한 보고서 발표\n28\nEU, 2024년 EIC 워크 프로그램 발표\n30\nOECD, 친환경 혁신을 위한 공동 창조와 대학의 역할\n32\n2. ICT\n35\n인도, 글로벌 기업 투자 유인하며 차세대 글로벌 제조 허브로 성장\n35\n차세대 지능형교통체계, LTE-V2X 통신방식으로 결정\n37\n구글, GPT-4 능가하는 멀티모달 성능의 ‘제미나이’ 공개\n39\n트위치, 게임 스트리밍 플랫폼 한국 서비스 6년 만에 종료\n41\n슈

In [38]:
vectorstore = Chroma.from_documents(
    splited_document,
    OpenAIEmbeddings(),
    collection_name = "science_trend",
    persist_directory="./chroma/"
)

In [39]:
mmr_retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [ ]:
# 값을 꼭 리턴해줘야함
@tool
def search_vector(query : str):
    """현재 과학 관련 정보를 제공"""
    rag = mmr_retriever.invoke(query)
    return rag

In [44]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model = ChatOpenAI(model = "gpt-4o-mini"),
    tools=[search_vector],
)


res = agent.invoke({"messages" :"한국 트위치 시장 종료 시점"})

In [46]:
pprint(res)

{'messages': [HumanMessage(content='한국 트위치 시장 종료 시점', additional_kwargs={}, response_metadata={}, id='c729c459-f21f-4d7f-a79a-ba37e7c777a6'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3VSBnOr1FZm29Ozbxhtkek6k', 'function': {'arguments': '{"query":"한국 트위치 시장 종료 시점"}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 52, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d1a67fdc-aa64-4624-8aa7-9456466e1d05-0', tool_calls=[{'name': 'search_vector', 'args': {'query': '한국 트위치 시장 종료 시점'}, 'id': 'call_3VSBnOr1FZm29Ozbxhtkek6k', 'type': 'tool_call'}], us